
1. Install packages
2. Import libraries
3. Load Phi-1.5
4. Manually preprocess with mask -100
5. Tokenize training and validation sets
6. Define TrainingArguments and Trainer
7. Fine-tune the model
8. Save the fine-tuned model
9. Load the fine-tuned model
10. Make normal predictions
11. Make chain-of-thought (COT) predictions
12. Save predictions to a .json file
13. Extract subsets with 100% correct predictions

In [1]:
#THIS IS FOR USING JUST ONE GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# 1. Instalation and import packages

In [2]:
!pip install transformers datasets evaluate

In [3]:
#Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch
import json
import re
from tqdm import tqdm

/home/maria/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Loading and Splitting the Dataset
 - We load the gsm8k dataset which contains grade-school math word problems:

- The official training split is splitt into 80% training and 10% validation sets.

- The provided test set is kept as is for final evaluation.

In [ ]:
# 1. Load and split GSM8K
gsm8k     = load_dataset("gsm8k", "main")
train_val = gsm8k["train"].shuffle(seed=42)
test_data = gsm8k["test"]                   

# 80-10 split of train_val
val_size      = int(0.1 * len(train_val))
split         = train_val.train_test_split(test_size=val_size, seed=42)
train_data    = split["train"]
val_data      = split["test"]

print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

Train: 6726, Val: 747, Test: 1319


In [5]:
# 2. Tokenizer & Model
model_checkpoint = "microsoft/phi-1_5" #Buscar un modelo compress de este
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    torch_dtype=torch.float32,
    device_map="auto"
)


# Usar GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)

# 3.  Preprocessing data manually

In [ ]:
# 3. Preprocessing with prompt-masking
max_length = 256
def preprocess_fn(examples):
    input_ids, attention_mask, labels = [], [], []
    for q, a in zip(examples["question"], examples["answer"]):
        prompt = f"Q: {q.strip()}\nA:"
        full   = prompt + " " + a.strip() + "\n"

        tok_full = tokenizer(
            full,
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )
        # mask the prompt tokens
        prompt_len = len(tokenizer(prompt)["input_ids"])
        label = [-100] * prompt_len + tok_full["input_ids"][prompt_len:]
        
        label = label + [-100] * (max_length - len(label))

        input_ids.append(tok_full["input_ids"])
        attention_mask.append(tok_full["attention_mask"])
        labels.append(label)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_train = train_data.map(
    preprocess_fn,
    batched=True,
    remove_columns=train_data.column_names,
)
tokenized_val = val_data.map(
    preprocess_fn,
    batched=True,
    remove_columns=val_data.column_names,
)

In [7]:
# 4. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


# 4. Define Trainging arguments and Train

In [ ]:
import torch, os
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

model.config.use_cache = False  # 
model.gradient_checkpointing_enable()


#  TrainingArguments & Trainer
training_args = TrainingArguments(
    output_dir="./results_phi",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=100,
    save_steps=1000,
    eval_steps=1000,
    save_total_limit=2,
    fp16= True,
    torch_compile=False,
    report_to=[],           
    run_name="gsm8k-finetune-phi" 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    # tokenizer=tokenizer   
)


# 6. Fine-tune
print("Starting fine-tuning…")
trainer.train()
trainer.save_model("./results_phi")

# 7. Load fine-tuned model for generation
model = AutoModelForCausalLM.from_pretrained("./results_phi")
model.to(device)
model.eval()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Starting fine-tuning…


Step,Training Loss
100,1.119400
200,1.005500
300,1.060400
400,1.076400
500,1.121900
600,1.133000
700,1.158300
800,1.139700
900,1.134800
1000,1.142400


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 27.69it/s]


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2048,), eps=1e-

# 5. Normal predictions

In [ ]:
import re
import pandas as pd
from tqdm.auto import tqdm

def extract_number(s):
    nums = re.findall(r"-?\d+\.?\d*", s)
    return nums[-1] if nums else None

results = []
for ex in tqdm(test_data, desc="Gen test set"):
    q, gold = ex["question"], ex["answer"].strip()
    prompt = f"Q: {q}\nA:"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    out = model.generate(
        **inputs,
        max_new_tokens=64,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id
    )

    gen_tokens = out[0][ inputs["input_ids"].shape[-1]: ]
    pred = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    gold_num = extract_number(gold)
    pred_num = extract_number(pred)
    correct  = (gold_num == pred_num)

    results.append({
        "question": q,
        "gold_full":    gold,
        "pred_full":    pred,
        "gold_number":  gold_num,
        "pred_number":  pred_num,
        "correct":      correct
    })

baseline_df  = pd.DataFrame(results)
baseline_acc = baseline_df["correct"].mean()
print(f" Baseline numeric-only accuracy: {baseline_acc:.3%}")


Gen test set: 100%|██████████| 1319/1319 [23:34<00:00,  1.07s/it]

 Baseline numeric-only accuracy: 15.239%


# 6. Save results of Normal predictions and perfect ones

In [ ]:
import json


with open("all_predictions_phi.json", "w") as f:
    json.dump(results, f, indent=2)

perfect = [r for r in results if r["correct"]]
with open("perfect_subset_phi.json", "w") as f:
    json.dump(perfect, f, indent=2)

print(f"Saved {len(results)} total preds and {len(perfect)} perfect ones.")


Saved 1319 total preds and 201 perfect ones.


Let's try with zero shot!

# 7. Predictions with COT ("Let's think step by step.")

In [ ]:
import re
from tqdm.auto import tqdm

def extract_number(s):
    
    nums = re.findall(r"-?\d+\.?\d*", s)
    return nums[-1] if nums else None

cot_results = []
for i, ex in enumerate(tqdm(test_data, desc="CoT test set")):
    q, raw_gold = ex["question"], ex["answer"]
    gold_num = extract_number(raw_gold)

    prompt = f"Q: {q}\nA: Let’s think step by step."
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(device)

    out = model.generate(
        **inputs,
        max_new_tokens=128,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id
    )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    
    if i < 5:
        print("––––––––––––––––––")
        print(text)
        print("gold_num:", gold_num)
        print("extracted:", extract_number(text))
        print("––––––––––––––––––\n")

    
    pred_num = extract_number(text)

    correct = (pred_num == gold_num)
    cot_results.append({
        "question": q,
        "gold_num": gold_num,
        "pred_num": pred_num,
        "correct": correct
    })

cot_acc = sum(r["correct"] for r in cot_results) / len(cot_results)
print(f" CoT zero-shot numeric accuracy: {cot_acc:.3%}")


CoT test set:   0%|          | 1/1319 [00:02<46:47,  2.13s/it]

––––––––––––––––––
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
A: Let’s think step by step. First, we need to find how many eggs remain after Janet eats 3 eggs for breakfast each day. We do this by subtracting 3 from the original number of eggs, performing 16 - 3 = <<16-3=13>>13 eggs remaining.
Next, we need to find how many eggs remain after Janet bakes muffins for her friends each day. We do this by subtracting 4 from the remaining number of eggs, performing 13 - 4 = <<13-4=9>>9 eggs remaining.
Now we need to find how much money Janet makes at the farmers' market by multiplying the remaining number of eggs by the
gold_num: 18
extracted: 9
––––––––––––––––––



CoT test set:   0%|          | 2/1319 [00:04<46:54,  2.14s/it]

––––––––––––––––––
Q: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
A: Let’s think step by step. We know that a robe takes 2 bolts of blue and half that much white, so it takes 2/2 = <<2/2=1>>1 bolt of white.
So in total, it takes 2 + 1 = <<2+1=3>>3 bolts.
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
#### 3
gold_num: 3
extracted: 3
––––––––––––––––––



CoT test set:   0%|          | 3/1319 [00:06<47:31,  2.17s/it]

...

––––––––––––––––––
Q: James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?
A: Let’s think step by step.
James runs 3*60=<<3*60=180>>180 meters a day.
So he runs 180*3=<<180*3=540>>540 meters a week.
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
#### 540
####
gold_num: 540
extracted: 540
––––––––––––––––––



CoT test set:   0%|          | 5/1319 [00:10<47:32,  2.17s/it]

––––––––––––––––––
Q: Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?
A: Let’s think step by step. Wendi gives her flock 15 cups in the morning and another 25 in the afternoon, for a total of 15+25=<<15+25=40>>40 cups of feed.
If she gives her 20 chickens 3 cups of feed in the morning and another 3 in the afternoon, she gives a total of 20+3=<<20+3=23>>23 cups of feed to her flock.
If she has a total of 40 cups of feed and she gives 23 cups in the morning and afternoon, she still needs to give her flock 40-23=<<40-23=17>>17 cups of feed in the
gold_num: 20
extracted: 17

CoT test set: 100%|██████████| 1319/1319 [47:03<00:00,  2.14s/it]

 CoT zero-shot numeric accuracy: 20.243%


# 8. Save results of COT predictions and perfect ones

In [ ]:
import json


with open("all_predictions_zeroCOT_phi.json", "w") as f:
    json.dump(cot_results, f, indent=2)

perfect = [r for r in cot_results if r["correct"]]
with open("perfect_subset_zoroCOT_phi.json", "w") as f:
    json.dump(perfect, f, indent=2)

print(f"Saved {len(cot_results)} total preds and {len(perfect)} perfect ones.")


Saved 1319 total preds and 267 perfect ones.
